In [ ]:
tplate = "/home/heatherm/type-I-ckine-model/ckine/brian/ti"
nkplate = "/home/heatherm/type-I-ckine-model/ckine/brian/ti"
tsample, titlesT = importF(tplate)
nksample, titlesNK = importF(nkplate)

dataT_array, pstatT_array, xfT_array, loadingT_array = pcaAll(tsample, "t", titlesT)
dataNK_array, pstatNK_array, xfNK_array, loadingNK_array = pcaAll(nksample, "n", titlesNK)